# 1/. Import Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

In [2]:
conn = sqlite3.connect("olist.sqlite")

View Data

In [3]:
# show all the table names
query = """
SELECT name
FROM sqlite_master
    WHERE type ='table'
      AND name NOT LIKE 'sqlite_%'
"""

pd.read_sql_query(query, conn)

,name
0,product_category_name_translation
1,sellers
2,customers
3,geolocation
4,order_items
5,order_payments
6,order_reviews
7,orders
8,products
9,leads_qualified


# 2/. EDA

2.1/. How the customers are distributed across the country?

In [4]:
query = """

SELECT
     geolocation_lat,
     geolocation_lng,
     customer_city,
     customer_state,
     COUNT(customer_id) AS count_customers
FROM customers AS c
     JOIN geolocation AS geo ON c.customer_zip_code_prefix = geo.geolocation_zip_code_prefix
GROUP BY 1, 2
ORDER BY 5 DESC
LIMIT 1000;

"""

customer_geo = pd.read_sql_query(query, conn)

customer_geo

,geolocation_lat,geolocation_lng,customer_city,customer_state,count_customers
0,-27.102099,-48.629613,itapema,SC,13816
1,-22.965906,-43.389999,rio de janeiro,RJ,9790
2,-22.907341,-43.103832,niteroi,RJ,8775
3,-22.905032,-43.114682,niteroi,RJ,8424
4,-22.904918,-43.097861,niteroi,RJ,7254
...,...,...,...,...,...
995,-20.260735,-40.271611,vitoria,ES,486
996,-24.306298,-46.986273,peruibe,SP,484
997,-23.009629,-43.423465,rio de janeiro,RJ,484
998,-23.009403,-43.422557,rio de janeiro,RJ,484


Insight: 
_ The highest numbers in Itapema and Rio de Janeiro
_ It appears that the majority of clients are dispersed throughout the eastern and southern regions of the nation. 
_ In the northwest of Brazil, we similarly observe a relatively sparse distribution of customers.

2.2/. How the sellers are distributed across the country?

In [5]:
query = """

SELECT
     geolocation_lat,
     geolocation_lng,
     seller_city,
     seller_state,
     COUNT(seller_id) AS count_sellers
FROM sellers AS s
     JOIN geolocation AS geo ON s.seller_zip_code_prefix = geo.geolocation_zip_code_prefix
GROUP BY 1, 2
ORDER BY 5 DESC
LIMIT 1000;

"""

seller_geo = pd.read_sql_query(query, conn)

seller_geo

,geolocation_lat,geolocation_lng,seller_city,seller_state,count_sellers
0,-27.102099,-48.629613,itapema,SC,314
1,-22.965906,-43.389999,rio de janeiro,RJ,267
2,-21.774868,-48.819540,ibitinga,SP,196
3,-21.768420,-48.836441,ibitinga,SP,196
4,-21.760595,-48.847152,ibitinga,SP,196
...,...,...,...,...,...
995,-23.623973,-46.666191,sao paulo,SP,22
996,-23.622356,-46.695400,sao paulo,SP,22
997,-23.612053,-46.607835,sao paulo,SP,22
998,-23.601396,-46.645933,sao paulo,SP,22


Insight:
_ We can see here that most of the sellers are located in and around the Sao Paulo region. 

=> Question: How these distributions might affect the delivery time and effectiveness?

# 3/. Deep Analysis

3.1/. The average delivery time by region

In [6]:
query = """

SELECT 
  customer_state, 
  ceil(AVG((JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_purchase_timestamp)) * 86400) / 86400) AS avg_delivery_days
FROM 
  (
    SELECT 
      customer_id, 
      order_delivered_customer_date, 
      order_purchase_timestamp
    FROM orders
    WHERE order_status='delivered'
  ) AS o
  LEFT JOIN customers AS c ON o.customer_id = c.customer_id
GROUP BY 1
ORDER BY 2;

"""
avg_region_delivery_time = pd.read_sql_query(query, conn)

avg_region_delivery_time

,customer_state,avg_delivery_days
0,SP,9.0
1,PR,12.0
2,DF,13.0
3,MG,13.0
4,SC,15.0
5,ES,16.0
6,GO,16.0
7,MS,16.0
8,RJ,16.0
9,RS,16.0


We see a strong relation between the number of sellers in the region and the average order delivery speed:
_ In the southeastern regions with a high concentration of sellers (Sao Paolo, Parana, Minas Gerais), the average delivery speed is less than two weeks. 
_ In the northern and northwestern regions (Amazonas, Anapa, Roraima), the average delivery time exceeds 25 days.

=> From this we can conclude that customers in high-revenue regions use the delivery service more frequently. 

=> Question: Let's see if the delivery cost relative to the total cost is higher or lower in those regions.

3.2/. State Average Freight Cost

In [7]:
query = """

SELECT 
    customer_state, 
    ROUND((SUM(freight_value) / SUM(price)) * 100, 2) as freight_cost_percent 
FROM 
    (
      SELECT 
          o.order_id, 
          o.customer_id, 
          customer_state, 
          price, 
          freight_value 
      FROM 
          orders AS o 
          LEFT JOIN order_items AS oi ON o.order_id = oi.order_id 
          LEFT JOIN customers AS c ON o.customer_id = c.customer_id
    ) AS t
GROUP BY 1
ORDER BY 2;

"""

state_avg_freight_cost = pd.read_sql_query(query, conn)

state_avg_freight_cost

,customer_state,freight_cost_percent
0,SP,13.81
1,MS,16.39
2,DF,16.73
3,RJ,16.75
4,MG,17.09
5,SC,17.22
6,PR,17.25
7,GO,18.03
8,RS,18.06
9,ES,18.09


Insight: 
The average share of shipping cost of the total order value is also lower in regions with high profitability and fast (relative to other regions) delivery.

3.3/. Top 5 Revenue State

In [8]:
query = """

SELECT 
    customer_state, 
    AVG(payment_value) AS avg_order_price, 
    SUM(payment_value) AS total_order_price 
FROM 
    (
        SELECT 
            o.order_id, 
            o.customer_id, 
            customer_state, 
            payment_value 
        FROM 
            orders AS o 
            LEFT JOIN order_payments AS op ON o.order_id = op.order_id 
            LEFT JOIN customers AS c ON o.customer_id = c.customer_id
    ) AS t
GROUP BY 1
ORDER BY 3 DESC;

"""

revenue_by_region = pd.read_sql_query(query, conn)

revenue_by_region

,customer_state,avg_order_price,total_order_price
0,SP,137.504630,5998226.96
1,RJ,158.525888,2144379.69
2,MG,154.706434,1872257.26
3,RS,157.180406,890898.54
4,PR,154.153626,811156.38
5,SC,165.979337,623086.43
6,BA,170.816017,616645.82
7,DF,161.134791,355141.08
8,GO,165.763404,350092.31
9,ES,154.706953,325967.55


In [9]:
top_5_states = revenue_by_region['customer_state'].head(5)

def categorize_state(state):
  return state if state in top_5_states.values else "Others"

revenue_by_region['customer_state'] = revenue_by_region['customer_state'].apply(categorize_state)

In [10]:
revenue_by_region

,customer_state,avg_order_price,total_order_price
0,SP,137.504630,5998226.96
1,RJ,158.525888,2144379.69
2,MG,154.706434,1872257.26
3,RS,157.180406,890898.54
4,PR,154.153626,811156.38
5,Others,165.979337,623086.43
6,Others,170.816017,616645.82
7,Others,161.134791,355141.08
8,Others,165.763404,350092.31
9,Others,154.706953,325967.55


Insight:
_ More than half of the total revenue was received from the Sao Paolo and Rio de Janeiro regions - which is consistent with the previous observation of a high concentration of buyers in these regions.

3.4/. AVG Order Price by Region

In [11]:
query = """

SELECT 
    customer_state, 
    AVG(payment_value) AS avg_order_price, 
    SUM(payment_value) AS total_order_price 
FROM 
    (
        SELECT 
            o.order_id, 
            o.customer_id, 
            customer_state, 
            payment_value 
        FROM 
            orders AS o 
            LEFT JOIN order_payments AS op ON o.order_id = op.order_id 
            LEFT JOIN customers AS c ON o.customer_id = c.customer_id
    ) AS t
GROUP BY 1
ORDER BY 3 DESC;

"""
avg_order_price_by_region = pd.read_sql_query(query, conn)

avg_order_price_by_region

,customer_state,avg_order_price,total_order_price
0,SP,137.504630,5998226.96
1,RJ,158.525888,2144379.69
2,MG,154.706434,1872257.26
3,RS,157.180406,890898.54
4,PR,154.153626,811156.38
5,SC,165.979337,623086.43
6,BA,170.816017,616645.82
7,DF,161.134791,355141.08
8,GO,165.763404,350092.31
9,ES,154.706953,325967.55


Insight: 
The average revenue in high-revenue regions (Sao Paolo, Rio de Janeiro, Minas Gerais, Parane) 
is significantly LOWER than the average order value in other regions.

3.5/. Customer satisfaction analysis

In [12]:
query = """

SELECT 
    customer_state, 
    AVG(review_score) AS avg_review_score 
FROM 
    (
      SELECT 
          o.order_id, 
          customer_state, 
          review_score 
      FROM 
          orders AS o 
          LEFT JOIN order_reviews AS re ON o.order_id = re.order_id 
          LEFT JOIN customers AS c ON o.customer_id = c.customer_id
    ) AS t
GROUP BY 1
ORDER BY 2 DESC;

"""
avg_review_score = pd.read_sql_query(query, conn)

avg_review_score

,customer_state,avg_review_score
0,AP,4.194030
1,AM,4.183673
2,PR,4.180032
3,SP,4.173951
4,MG,4.136172
5,RS,4.133321
6,MS,4.118785
7,RN,4.105809
8,MT,4.102990
9,TO,4.096774


Insight: Customers are most satisfied with the quality of service in the most "active" regions: Sao Paolo, Minas Gerais, Parana). Order estimates in Roraima clearly indicate problems with the quality of service in the region.

# 4/. Conclusion

Areas in the south and southeast of the nation: 
_ Highly profitable
_ Have a concentration of customers who buy a lot and frequently
_ Receive their goods fast, pay relatively little for shipping, and are generally happy with the level of service. 
_ Sao Paolo, Minas Gerais, and Parana are the three regions in this cluster with the highest performance indicators.

Regions in the country's north and northwest: 
_ Have low concentration of buyers and sellers, as well as ones with high costs and lengthy delivery times. 
_ Roraima, Alagoas, and Maranhao are the three most troublesome regions in this cluster and may be easily differentiated. 
_ In these areas, the average delivery time exceeds three weeks. 
_ Also, compared to other regions, practically all of these orders obtain reviews with rather poor scores.

# Export Data (Tableau)

In [13]:
# customer_geo.to_excel("customer_geo.xlsx")

In [14]:
# seller_geo.to_excel("seller_geo.xlsx")

In [15]:
# avg_region_delivery_time.to_excel("avg_region_delivery_time.xlsx")

In [16]:
# state_avg_freight_cost.to_excel("state_avg_freight_cost.xlsx")

In [17]:
# revenue_by_region.to_excel("revenue_by_region.xlsx")

In [18]:
# avg_review_score.to_excel("avg_review_score.xlsx")